In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:

import os
import json
from datetime import datetime

import requests
from bs4 import BeautifulSoup
from lxml import html
import xml.etree.ElementTree as ET
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

import pandas as pd
import xlrd
import openpyxl

import pprint

import warnings
warnings.filterwarnings(action='ignore')

KEY =  E01BZDE021JEBJM8P5XP

In [46]:
# 한국은행 통계시스템 API로 부터 데이터 요청 하는 함수
def get_data_from_bok_api(KEY, STAT_CD, PERIOD, STAT_DATE, END_DATE):
  url = 'http://ecos.bok.or.kr/api/StatisticSearch/{}/xml/kr/1/30000/{}/{}/{}/{}'\
.format(
          KEY        # 인증키
        , STAT_CD    # 추출할 통계지표의 코드
        , PERIOD     # 기간 단위
        , STAT_DATE  # 데이터 시작일
        , END_DATE   # 데이터 종료일
        )
  response =  requests.get(url)
  if response.status_code == 200:
    try:
      contents = response.text
      ecosRoot = ET.fromstring(contents)
      if ecosRoot[0].text[:4] in ("INFO","ERRO"):
        print(ecosRoot[0].text + " : " + ecosRoot[1].text)
      else:
        re = response.content.decode('utf-8')
        xml_obj = BeautifulSoup(re, 'lxml-xml')
        rows = xml_obj.findAll("row")
        return rows
    except Exception as e:
      print(str(e))


In [70]:
data_dict = {
    '1400' :  '국내총생산(시장가격, GDP)'
}

KEY = 'E01BZDE021JEBJM8P5XP'

item_list = [
     'STAT_CODE'
    ,'STAT_NAME'
    ,'ITEM_CODE1'
    ,'ITEM_NAME1'
    # ,'ITEM_CODE2'
    # ,'ITEM_NAME2'
    # ,'ITEM_CODE3'
    # ,'ITEM_NAME3'
    ,'UNIT_NAME'
    ,'TIME'
    ,'DATA_VALUE'
]
PERIOD = 'M'
STAT_DATE = '202001'
END_DATE =  '202301'

result_list = list()

for k in data_dict.keys():
  rows = get_data_from_bok_api(KEY, k, PERIOD, STAT_DATE, END_DATE)
  print(len(rows))

  for p in range(0, len(rows)):
    info_list = list()

    for i in item_list:
      try:
        individual_info = rows[p].find(i).text
      except:
        individual_info = ""
      info_list.append(individual_info)
    result_list.append(info_list)
  result_list


INFO-200 : 해당하는 데이터가 없습니다.


TypeError: ignored

In [48]:
result_df = pd.DataFrame(result_list, columns=[
     'STAT_CODE'
    ,'STAT_NAME'
    ,'ITEM_CODE1'
    ,'ITEM_NAME1'
    ,'ITEM_CODE2'
    ,'ITEM_NAME2'
    ,'ITEM_CODE3'
    ,'ITEM_NAME3'
    ,'UNIT_NAME'
    ,'TIME'
    ,'DATA_VALUE'
]).drop_duplicates()

result_df

,STAT_CODE,STAT_NAME,ITEM_CODE1,ITEM_NAME1,ITEM_CODE2,ITEM_NAME2,ITEM_CODE3,ITEM_NAME3,UNIT_NAME,TIME,DATA_VALUE
0,521Y001,6.4. 뉴스심리지수(실험적 통계),A001,뉴스심리지수,,,,,,20230612,103.14


In [ ]:
date =  datetime.today().strftime('%Y-%m-%d')
result_df.drop_duplicates().to_excel(date+'_korea bank indicators.xlsx', index = False)

In [67]:
data_path = '/content/drive/MyDrive/ZZ/Data/' # './'
data_list = [file for file in os.listdir(data_path) if file.endswith(".xlsx")]
i = -1
print(data_list[-1])
data = pd.read_excel(os.path.join(data_path,data_list[i]),header=1)
data.head()

f_경제활동별 GDP 및 GNI(원계열, 명목, 분기 및 연간).xlsx


,코드,1101,110101,110102,110103,110104,1102,1103,110301,110302,...,1111,1112,1113,11131,11132,1200,1300,1400,1500,1600
0,계정항목,농림어업,재배업,축산업,임업,어업,광업,제조업,음식료품 제조업,섬유 및 가죽제품 제조업,...,교육서비스업,"의료, 보건업 및 사회복지서비스업",문화 및 기타 서비스업,"예술, 스포츠 및 여가관련 서비스업",기타 서비스업,총부가가치(기초가격),순생산물세,"국내총생산(시장가격, GDP)",국외순수취요소소득,국민총소득(GNI)
1,단위,십억원,십억원,십억원,십억원,십억원,십억원,십억원,십억원,십억원,...,십억원,십억원,십억원,십억원,십억원,십억원,십억원,십억원,십억원,십억원
2,변환,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,...,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료
3,2023/Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023/Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
data.head(10)

,코드,1101,110101,110102,110103,110104,1102,1103,110301,110302,...,1111,1112,1113,11131,11132,1200,1300,1400,1500,1600
0,계정항목,농림어업,재배업,축산업,임업,어업,광업,제조업,음식료품 제조업,섬유 및 가죽제품 제조업,...,교육서비스업,"의료, 보건업 및 사회복지서비스업",문화 및 기타 서비스업,"예술, 스포츠 및 여가관련 서비스업",기타 서비스업,총부가가치(기초가격),순생산물세,"국내총생산(시장가격, GDP)",국외순수취요소소득,국민총소득(GNI)
1,단위,십억원,십억원,십억원,십억원,십억원,십억원,십억원,십억원,십억원,...,십억원,십억원,십억원,십억원,십억원,십억원,십억원,십억원,십억원,십억원
2,변환,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,...,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료
3,2023/Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023/Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2023/Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023/Q1,6457.3,4435.6,1245,180.2,596.5,535.4,125497,7173.6,4181.4,...,26363,27606.9,12986,6729.2,6256.8,482049.2,41767,523816.2,17206.4,541022.6
7,2022/Q4,10270.3,7919.8,950.1,790.8,609.7,477.2,132857.3,6665.4,3509.4,...,24544,26915.8,12086.9,5329.9,6757,510019.5,46609.2,556628.7,12786.7,569415.4
8,2022/Q3,8481,5982,1307.9,692.2,499,403.3,144165.9,5651.2,2745.5,...,26459.4,25805.4,13953.8,7172,6781.9,499393.8,49498.7,548892.5,11055.5,559948
9,2022/Q2,9657.3,5767.4,2560.7,768.2,561,512.5,143566.9,6616.5,3277,...,25813.9,26598.2,10726.8,4480.6,6246.3,497403,46836.7,544239.8,2264.3,546504
